In [ ]:
using CairoMakie

import LilGuys as lguys
using Arya

In [ ]:
import TOML

In [ ]:
using Measurements

In [ ]:
function sigma_axis(gp; kwargs...) 
	ax = Axis(gp, 
		xlabel=log_r_label,
		ylabel = L"\log \Sigma\ / \textrm{(stars/arcmin^2)}",
        limits=(-1, 2, -2, 2.5)
		;kwargs...
	)

	return fig, ax
end

In [ ]:
prof_expected = lguys.StellarProfile("/astro/dboyea/dwarfs/observations/sculptor/processed/fiducial_sample_profile.toml");

In [ ]:
log_r_label = "log r / arcmin"
log_sigma_label = L"$\log \Sigma$ / N\,arcmin$^{-2}$"

In [ ]:
function plot_finals(models)
    
    fig = Figure()
    Axis(fig[1, 1], 
        xlabel = log_r_label, 
        ylabel = log_sigma_label,
        limits = (-0.5, 2.3, -2, 2)
        )

	errscatter!(prof_expected.log_r, prof_expected.log_Sigma,
		yerr=prof_expected.log_Sigma_err,
		color=:black,
        label="J+24",
	)

    for (label, model) in models
    	lines!(model.log_r, model.log_Sigma, 
    			label=label)
    end

	axislegend()

	fig
end

In [ ]:
function plot_ini_fin(profs, model, normalization=0; 
        limits=(-0.5, 2.3, -2, 2),
        r_b = NaN,
        kwargs...
    )
    
    fig = Figure()
    Axis(fig[1, 1];
        xlabel = log_r_label, 
        ylabel = log_sigma_label,
        limits = limits, 
        kwargs...
    )

	errscatter!(prof_expected.log_r, prof_expected.log_Sigma,
		yerr=prof_expected.log_Sigma_err,
		color=:black,
        label="J+24",
	)

    for (label, prof) in profs
        lines!(prof.log_r, prof.log_Sigma .+ normalization, 
                label=label)
    end


    if model !== nothing

        x = LinRange(-2, 2.5, 1000)
        r = 10 .^ x
        y = lguys.calc_Σ.(model, r)
    
        r_scale = lguys.kpc_to_arcmin(1, profs[1].second.distance)
    
    
        x = x .+ log10(r_scale)
        y .*= 1 / r_scale ^ 2
    
        lines!(x, log10.(y) .+ normalization, linestyle=:dot, color=:black, label="expected")

    end
    
    if r_b !== NaN
        dy = limits[4]-limits[3]
        dx = limits[2] - limits[1]
        
        x0 = limits[1]
        y0 = limits[3]

        a = log10(r_b)
        _, prof = profs[end]
        idx = argmin(abs.(prof.log_r .- a))
        b = prof.log_Sigma[idx] + normalization .+ 0.1dy
        va = 0
        vb = -0.05dy
        
        arrows!([a], [b], [va], [vb])
        println(a, ", ", b)
        text!(a, b, text=L"r_b")

    end

	axislegend(position=:lb)

	fig
end

In [ ]:
function get_log_Σ_0(prof, χ2_max=1)

    local log_Σ_0, log_Σ_0_err

    pre_filt = isfinite.(prof.log_Sigma)
    log_r = prof.log_r[pre_filt]
    log_Σ = prof.log_Sigma[pre_filt]
    log_Σ_err = prof.log_Sigma_err[pre_filt]
    @assert issorted(log_r)

    for i in eachindex(log_r)
        filt = 1:i
        log_Σ_0 = lguys.mean(log_Σ[filt] .± log_Σ_err[filt])
        log_Σ_0_err = Measurements.uncertainty.(log_Σ_0)
        log_Σ_0 = Measurements.value(log_Σ_0)
        χ2 = @. (log_Σ_0 - log_Σ)^2 / (log_Σ_err + log_Σ_0_err)^2

        χ2_norm = lguys.mean(χ2[filt])

        if χ2_norm .> χ2_max
            @info "max radius of $(log_r[i])"
            break
        end
    end
    
    return log_Σ_0, log_Σ_0_err
end

In [ ]:
function get_normalization(prof_f, prof_expected=prof_expected)
    log_Σ_0_obs, _ = get_log_Σ_0(prof_expected)
    log_Σ_0, _ = get_log_Σ_0(prof_f)

    return log_Σ_0_obs - log_Σ_0
end

In [ ]:
modelsdir = ENV["DWARFS_ROOT"] * "/analysis/"

In [ ]:
function read_profiles(filename, starsname)
    
    prof_iso_i = lguys.StellarProfile(modelsdir * filename * "/stars/$starsname/iso_initial_profile.toml")
    prof_iso_p = lguys.StellarProfile(modelsdir * filename * "/stars/$starsname/iso_paint_profile.toml")
    prof_iso_f = lguys.StellarProfile(modelsdir * filename * "/stars/$starsname/iso_final_profile.toml")
    
    prof_i = lguys.StellarProfile(modelsdir * filename * "/stars/$starsname/initial_profile.toml")
    prof_f = lguys.StellarProfile(modelsdir * filename * "/stars/$starsname/final_profile.toml")
    prof_ana = lguys.load_profile(modelsdir * filename * "/../stars/$starsname/profile.toml")

    
    profs = [
        "iso initial" => prof_iso_i,
        "stars initial" => prof_iso_p,
        "iso final" => prof_iso_f,
        "orbit initial" => prof_i,
        "orbit final" => prof_f,
    ]

    return profs, prof_ana
end

In [ ]:
function read_profiles_simple(filename, starsname)
    
    prof_iso_p = lguys.StellarProfile(modelsdir * filename * "/stars/$starsname/iso_paint_profile.toml")
    prof_f = lguys.StellarProfile(modelsdir * filename * "/stars/$starsname/final_profile.toml")

    profs = [
        "initial" => prof_iso_p,
        "final" => prof_f,
    ]

    return profs
end

# 1e7 

## Compact halo, mean orbit

In [ ]:
modeldir = "sculptor/1e7_V31_r3.2/orbit_mean"

In [ ]:
orbit_props = TOML.parsefile(joinpath(modeldir, "orbital_properties.toml"))

### Exp2d(Rs=0.10)

In [ ]:
profs, expected = read_profiles(modeldir, "exp2d_rs0.10");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = lguys.calc_break_radius(prof_f.sigma_v / lguys.V2KMS, orbit_props["t_last_peri"] / lguys.T2GYR)
r_b_arcmin = lguys.kpc_to_arcmin(r_b, orbit_props["distance_f"])


In [ ]:
plot_ini_fin(profs[1:5], expected, norm, limits=(-1, 3, -10, 2), r_b=r_b_arcmin)



### Exp2d(Rs=0.13)

In [ ]:
profs, expected = read_profiles(modeldir, "exp2d_rs0.13");

In [ ]:
profs = read_profiles_simple(modeldir, "exp2d_rs0.13");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = lguys.calc_break_radius(prof_f.sigma_v / lguys.V2KMS, orbit_props["t_last_peri"] / lguys.T2GYR)
r_b_arcmin = lguys.kpc_to_arcmin(r_b, orbit_props["distance_f"])


In [ ]:
plot_ini_fin(profs, nothing, norm, limits=(-1, 3, -10, 2), r_b=r_b_arcmin,
    title="Exp2D(Rs=0.13 kpc)",
)



In [ ]:
plot_ini_fin(profs[1:5], expected, norm, limits=(-1, 3, -10, 2), r_b=r_b_arcmin,
    title="Exp2D(Rs=0.13 kpc)",
)



### Exp2d, Rs=0.15

In [ ]:
profs, expected = read_profiles(modeldir, "exp2d_rs0.15");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = lguys.calc_break_radius(prof_f.sigma_v / lguys.V2KMS, orbit_props["t_last_peri"] / lguys.T2GYR)
r_b_arcmin = lguys.kpc_to_arcmin(r_b, orbit_props["distance_f"])


In [ ]:
plot_ini_fin(profs[1:5], expected, norm, limits=(-1, 3, -10, 2), r_b=r_b_arcmin)



### plummer

In [ ]:
profs = read_profiles_simple(modeldir, "plummer_rs0.20");

In [ ]:
profs, expected = read_profiles(modeldir, "plummer_rs0.20");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = lguys.calc_break_radius(prof_f.sigma_v / lguys.V2KMS, orbit_props["t_last_peri"] / lguys.T2GYR)
r_b_arcmin = lguys.kpc_to_arcmin(r_b, orbit_props["distance_f"])


In [ ]:
fig = plot_ini_fin(profs, expected, norm, limits=(-1, 3, -6, 2), r_b=r_b_arcmin, title="Plummer(Rs=0.20kpc)")



In [ ]:
prof_1e7 = profs[end]

In [ ]:
plot_ini_fin(profs[1:5], expected, norm, limits=(-1, 3, -6, 2), r_b=r_b_arcmin, title="Plummer(Rs=0.20kpc)")



# 1e6

## Mean mean

In [ ]:
modeldir = "sculptor/1e6_V31_r3.2/orbit1"

In [ ]:
orbit_props = TOML.parsefile(joinpath(modeldir, "orbital_properties.toml"))

### Plummer 0.15

In [ ]:
profs, expected = read_profiles(modeldir, "plummer_rs0.15");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = lguys.calc_break_radius(prof_f.sigma_v / lguys.V2KMS, orbit_props["t_last_peri"] / lguys.T2GYR)
r_b_arcmin = lguys.kpc_to_arcmin(r_b, orbit_props["distance_f"])

In [ ]:
plot_ini_fin(profs, expected, norm, limits=(-1, 3, -6, 2), r_b=r_b_arcmin)

### Plummer 0.20

In [ ]:
profs, expected = read_profiles(modeldir, "plummer_rs0.20");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = lguys.calc_break_radius(prof_f.sigma_v / lguys.V2KMS, orbit_props["t_last_peri"] / lguys.T2GYR)
r_b_arcmin = lguys.kpc_to_arcmin(r_b, orbit_props["distance_f"])

In [ ]:
plot_ini_fin(profs, expected, norm - 0.1, limits=(-1, 3, -6, 2), r_b=r_b_arcmin)

## SmallPeri

In [ ]:
modeldir = "sculptor/1e6_V31_r3.2/orbit_smallperi"

In [ ]:
orbit_props = TOML.parsefile(joinpath(modeldir, "orbital_properties.toml"))

### Exponential

In [ ]:
profs, expected = read_profiles(modeldir, "exp2d_rs0.10");

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = lguys.calc_break_radius(prof_f.sigma_v / lguys.V2KMS, orbit_props["t_last_peri"] / lguys.T2GYR)
r_b_arcmin = lguys.kpc_to_arcmin(r_b, orbit_props["distance_f"])

In [ ]:
plot_ini_fin(profs, expected, norm, limits=(-1, 3, -10, 2), r_b=r_b_arcmin)

### plummer

In [ ]:
profs, expected = read_profiles(modeldir, "plummer_rs0.20");

In [ ]:
profs[1].second.M_in[profs[1].second.log_r .> 2]

In [ ]:
profs[end].second.M_in[profs[end].second.log_r .> 2]

In [ ]:
prof_f = profs[end].second
norm = get_normalization(prof_f)

r_b = lguys.calc_break_radius(prof_f.sigma_v / lguys.V2KMS, orbit_props["t_last_peri"] / lguys.T2GYR)
r_b_arcmin = lguys.kpc_to_arcmin(r_b, orbit_props["distance_f"])

In [ ]:
prof_1e7 = "mean orbit final (1e7)" => prof_1e7.second

In [ ]:
append!(profs, [prof_1e7])

In [ ]:
plot_ini_fin(profs, expected, norm, limits=(-1, 3, -6, 2), r_b=r_b_arcmin)